<a href="https://colab.research.google.com/github/lingo-mit/6864-hw1/blob/master/6864_hw1b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
# %%bash
# !(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit 
# rm -rf 6864-hw1
# git clone https://github.com/lingo-mit/6864-hw1.git

Cloning into '6864-hw1'...


In [2]:
import sys
sys.path.append("/content/6864-hw1")

import csv
import itertools as it
import numpy as np
np.random.seed(0)

from scipy.special import logsumexp
import lab_util

## Hidden Markov Models

In the remaining part of the lab (containing part 3) you'll use the Baum--Welch algorithm to learn _categorical_ representations of words in your vocabulary. Answers to questions in this lab should go in the same report as the initial release.

As before, we'll start by loading up a dataset:

In [4]:
data = []
n_positive = 0
n_disp = 0
with open("reviews.csv") as reader:
  csvreader = csv.reader(reader)
  next(csvreader)
  for id, review, label in csvreader:
    label = int(label)

    # hacky class balancing
    if label == 1:
      if n_positive == 2000:
        continue
      n_positive += 1
    if len(data) == 4000:
      break

    data.append((review, label))
    
    if n_disp > 5:
      continue
    n_disp += 1
    print("review:", review)
    print("rating:", label, "(good)" if label == 1 else "(bad)")
    print()

print(f"Read {len(data)} total reviews.")
np.random.shuffle(data)
reviews, labels = zip(*data)
train_reviews = reviews[:3000]
train_labels = labels[:3000]
val_reviews = reviews[3000:3500]
val_labels = labels[3000:3500]
test_reviews = reviews[3500:]
test_labels = labels[3500:]

review: I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.
rating: 1 (good)

review: Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".
rating: 0 (bad)

review: This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar.  And it is a tiny mouthful of heaven.  Not too chewy, and very flavorful.  I highly recommend this yummy treat.  If you are familiar with the story of C.S. Lewis' "The Lion, The Witch, and The Wardrobe" - this is the treat that seduces Edmund into selling out his Brother an

Next, implement the forward--backward algorithm for HMMs like we saw in class.

**IMPORTANT NOTE**: if you directly multiply probabilities as shown on the class slides, you'll get underflow errors. You'll probably want to work in the log domain (remember that `log(ab) = log(a) + log(b)`, `log(a+b) = logaddexp(a, b)`).

In [6]:
# hmm model
class HMM(object):
    def __init__(self, num_states, num_words):
        self.num_states = num_states
        self.num_words = num_words

        self.states = range(num_states)
        self.symbols = range(num_words)

        # initialize the matrix A with random transition probabilities p(j|i)
        # A should be a matrix of size `num_states x num_states`
        # with rows that sum to 1
        # your code here
        #self.A = np.random.rand(num_states, num_states)
        #for i in range(num_states):
            #self.A[i,:]/= self.A[i,:].sum()
        #print(self.A.shape)
        self.A = np.random.dirichlet(np.ones(num_states), num_states)
        # initialize the matrix B with random emission probabilities p(o|i)
        # B should be a matrix of size `num_states x num_words`
        # with rows that sum to 1
        #self.B = np.random.rand(num_states, num_words)
        #for i in range(num_states):
            #self.B[i,:]/= self.B[i,:].sum()
        #print(self.B.shape)
        self.B = np.random.dirichlet(np.ones(num_words), num_states)
            
        # initialize the vector pi with a random starting distribution
        # pi should be a vector of size `num_states`
        # your code here
        #self.pi = np.random.rand(num_states)
        #self.pi /= self.pi.sum()
        self.pi = np.random.dirichlet(np.ones(num_states), 1).flatten()
        #print(self.pi.shape)

    def generate(self, n):
        """randomly sample the HMM to generate a sequence.
        """
        # we'll give you this one

        sequence = []
        # initialize the first state
        state = np.random.choice(self.states, p=self.pi)
        for i in range(n):
            # get the emission probs for this state
            b = self.B[state, :]
            # emit a word
            word = np.random.choice(self.symbols, p=b)
            sequence.append(word)
            # get the transition probs for this state
            a = self.A[state, :]
            # update the state
            state = np.random.choice(self.states, p=a)
        return sequence

    def forward(self, obs):
        # run the forward algorithm
        # this function should return a `len(obs) x num_states` matrix
        # where the (i, j)th entry contains p(obs[:t], hidden_state_t = i)
        
        log_A = np.log(self.A)
        log_B = np.log(self.B)
        log_pi = np.log(self.pi)
        alpha = np.zeros((len(obs), self.num_states))
        # your code here!
        #alpha[0,:] = self.pi*self.B[:,obs[0]]
        #alpha[0,:] = log_pi + log_B[:,obs[0]]
        for i in range(self.num_states):
            #alpha[0,i] = self.pi[i]*self.B[i,obs[0]]
            alpha[0,i] = log_pi[i] + log_B[i, obs[0]]
            
        for i in range(1, len(obs)):
            for j in range(self.num_states):
                #sum_alpha_a = alpha[i-1,:].dot(self.A[:,j])
                sum_alpha_a = logsumexp(alpha[i-1,:] + log_A[:,j])
                #alpha[i,j] = sum_alpha_a*self.B[j,obs[i]]
                alpha[i,j] = sum_alpha_a + log_B[j,obs[i]]
            #alpha[i,:] = alpha[i,:]/alpha[i,:].sum()
        
        #alpha += 1e-300
        #return np.log(alpha)
        return alpha

    def backward(self, obs):
        # run the backward algorithm
        # this function should return a `len(obs) x num_states` matrix
        # where the (i, j)th entry contains p(obs[t+1:] | hidden_state_t = i)
        
        beta = np.log(np.ones((len(obs), self.num_states)))
        #beta1 = np.log(np.ones((len(obs), self.num_states))) 
        log_A = np.log(self.A)
        log_B = np.log(self.B)
        log_pi = np.log(self.pi)
        
        # your code here!
        
        for i in range(len(obs)-2, -1, -1):
            for j in range(self.num_states):
                #beta[i, j] = (self.A[j,:]*beta[i+1,:]).dot(self.B[:,obs[i+1]])
                beta[i,j] = logsumexp(log_A[j,:] + beta[i+1,:] + log_B[:,obs[i+1]])
            #beta[i,:] = beta[i,:]/beta[i,:].sum()
        
        #for i in range(len(obs)-2, -1, -1):
            #for j in range(self.num_states):
                #beta[i, j] = (self.A[j,:]*beta[i+1,:]).dot(self.B[:,obs[i+1]])
            
                
        #beta += 1e-300
        #return np.log(beta)
        return beta
        
    def forward_backward(self, obs):
        # compute forward--backward scores

        # logprob is the total log-probability of the sequence obs 
        # (marginalizing over hidden states)

        # gamma is a matrix of size `len(obs) x num_states`
        # it contains the marginal probability of being in state i at time t

        # xi is a tensor of size `len(obs) x num_states x num_states`
        # it conains the marginal probability of transitioning from i to j at t

        log_A = np.log(self.A)
        log_B = np.log(self.B)
        log_pi = np.log(self.pi)
        
        # your code here!
        alpha = self.forward(obs)
        beta = self.backward(obs)
        
        #prob_o_lambda = alpha[-1,:].sum()
        logprob = logsumexp(alpha[-1,:])
        #logprob = logsumexp(log_pi + log_B[:,0].T + beta[0,:])
        
        xi = np.zeros((len(obs), self.num_states, self.num_states))
        gamma = np.zeros((len(obs), self.num_states))
        #print(gamma.shape)
        
        for i in range(len(obs)-1):
            for j in range(self.num_states):
                #gamma[i,j] = alpha[i,j]*beta[i,j]/sum_gamma
                for k in range(self.num_states):
                    #xi[i,j,k] = (alpha[i,j]*self.A[j,k]*self.B[k,obs[i+1]]*beta[i+1,k])/prob_o_lambda
                    #x[i,j,k] = alpha(i,j) + self.A[j,k] + self.B[k,obs[i+1]] + beta(i+1,k) - logprob
                    xi[i,j,k] = alpha[i,j] + log_A[j,k] + log_B[k,obs[i+1]] + beta[i+1,k] - logprob
        
        gamma = alpha + beta - logprob
            
        return logprob, xi, gamma

    def learn_unsupervised(self, corpus, num_iters):
        """Run the Baum Welch EM algorithm
        """
        print(len(corpus))
        temp_list = []
        for i_iter in range(num_iters):
            print(i_iter)
            expected_si = np.zeros(self.num_states) # your code here
            expected_sj = np.zeros(self.num_states)
            expected_sij = np.zeros((self.num_states, self.num_states)) # your code here
            expected_sjwk = np.zeros((self.num_states, self.num_words))
            total_logprob = 0
            
            #print('A')
            #print(self.A)
            #print('B')
            #print(self.B)
            #print(np.sum(self.B, axis = 1))
            #print('pi')
            #print(self.pi)
            
            pi_new = np.zeros(self.num_states)         
            num_review = 0
            
            for review in corpus:
                #print(str(num_review))
                num_review += 1
                logprob, xi, gamma = self.forward_backward(review)
                # your code here
                exp_xi = np.exp(xi)
                exp_gamma = np.exp(gamma)
                #print(exp_gamma)
                #total_logprob = np.logaddexp(total_logprob, logprob)
                total_logprob += logprob
                expected_si = expected_si + np.sum(exp_gamma[:-1,:], axis = 0)
                expected_sj = expected_sj + np.sum(exp_gamma, axis = 0)
                expected_sij = expected_sij + np.sum(exp_xi[:-1,:,:], axis = 0)
                
                #expected_sjwk_review = np.zeros((self.num_states, self.num_words))
                for idx in range(len(review)):
                    #for s in range(self.num_states):
                    expected_sjwk[:,review[idx]] = expected_sjwk[:,review[idx]] + exp_gamma[idx,:].T
                #for j in range(self.num_states):
                    #expected_si[j] = np.logaddexp(expected_si[j], logsumexp(gamma[:-1,j]))
                    #expected_sj[j] = np.logaddexp(expected_sj[j], logsumexp(gamma[:,j]))
                    #for k in range(self.num_states):
                        #expected_sij[j,k] = np.logaddexp(expected_sij[j,k], logsumexp(xi[:-1,j,k]))
                                       
                #for i in range(len(review)):
                    #expected_sjwk[:,review[i]] = np.logaddexp(expected_sjwk[:,review[i]], np.transpose(gamma[i,:]))
                
                pi_new += exp_gamma[0,:]
            
            temp_list.append(total_logprob)
            print("log-likelihood", total_logprob)
            
            if i_iter > 0:
                print(abs(temp_list[i_iter]/temp_list[i_iter - 1] - 1))
                if abs(temp_list[i_iter]/temp_list[i_iter - 1] - 1) < 1e-4:
                    break
                
            A_new = np.zeros((self.num_states, self.num_states)) # your code here
            B_new = np.zeros((self.num_states, self.num_words)) # your code here
            pi_new = pi_new/len(corpus)
            
            for i in range(self.num_states):
                A_new[i,:] = expected_sij[i,:]/expected_si[i]
                B_new[i,:] = expected_sjwk[i,:]/expected_sj[i]
                    #A_new[i,j] = expected_sij[i,j] - expected_si[i]
                                                                              
            self.A = A_new
            self.B = B_new
            self.pi = pi_new 

Train a model:

In [7]:
tokenizer = lab_util.Tokenizer()
tokenizer.fit(train_reviews)
train_reviews_tk = tokenizer.tokenize(train_reviews)
print(tokenizer.vocab_size)

hmm = HMM(num_states=10, num_words=tokenizer.vocab_size)
log_alpha = hmm.forward(train_reviews_tk[1])
#print('alpha')
#print(log_alpha)
log_beta = hmm.backward(train_reviews_tk[1])
#print('beta')
#print(log_beta)
#log_prob, xi, gamma = hmm.forward_backward(train_reviews_tk[2])
#print('gamma')
#print(log_alpha + log_beta - log_prob)
#print('log_prob')
#print(log_prob)
#print(xi.shape)
hmm.learn_unsupervised(train_reviews_tk, 3)

2006
3000
0
log-likelihood -2101804.921020325
1
log-likelihood -1524236.8993465118
0.2747962077248499
2
log-likelihood -1521976.7071049106
0.0014828352748645912


Let's look at some of the words associated with each hidden state:

In [213]:
for i in range(hmm.num_states):
    most_probable = np.argsort(hmm.B[i, :])[:10]
    print(f"state {i}")
    for o in most_probable:
        print(tokenizer.token_to_word[o], hmm.B[i, o])
    print()

state 0
www 8.644481953986294e-45
http 1.7180692889504047e-42
suppose 9.680339437106044e-36
marzano 2.9985202824174896e-34
gp 2.1749795483487064e-31
tend 1.4391562299843026e-29
able 2.336634017041316e-29
moved 8.378941790468853e-29
hoped 1.0265405340036787e-27
decided 9.409025624544188e-27

state 1
marzano 1.7719159471921054e-52
www 4.4854344888031915e-41
gp 7.441701195224051e-37
http 1.554690438322806e-33
suppose 2.8824713807736332e-27
am 8.583263494025701e-25
son's 1.8480420691818067e-24
splash 1.3228306660557773e-22
figured 3.076055841538382e-22
spent 6.104758881781126e-22

state 2
gp 1.4497052198202335e-31
0g 1.843371532450621e-26
www 2.205823505639042e-25
concentrates 1.3924720263560608e-21
agave 1.695486912144952e-21
http 3.272955807604809e-21
puck 1.0589084397216047e-20
splash 6.898083121263165e-20
nectar 8.1374679796819e-20
dijon 1.982865450320652e-18

state 3
gp 3.708780741216216e-45
www 2.670569709561586e-35
http 3.734085286686433e-32
grey 7.16115913799756e-23
son's 7.1449756

We can also look at some samples from the model!

In [214]:
for i in range(10):
    print(tokenizer.de_tokenize([hmm.generate(10)]))

['i <unk> control no <unk> seem that actual delicious on']
['no <unk> sodas orange food these any best from it']
['one . <unk> coconut . <unk> tea organic hot of']
['itself the amazon because so <unk> us im started the']
['brands , . the the br no br amazon <unk>']
["hoping <unk> value . , . but i'm on fine"]
['as think decided wanted delicious was with many or really']
['great gum store such savory bit problems added for the']
['it taste ! used the <unk> br <unk> iron ,']
['diet cup in up <unk> on find cooked if and']


Finally, let's repeat the classification experiment from Parts 1 and 2, using the _vector of expected hidden state counts_ as a sentence representation.

(Warning! results may not be the same as in earlier versions of this experiment.)

In [13]:
def train_model(xs_featurized, ys):
  import sklearn.linear_model
  model = sklearn.linear_model.LogisticRegression()
  model.fit(xs_featurized, ys)
  return model

def eval_model(model, xs_featurized, ys):
  pred_ys = model.predict(xs_featurized)
  print("test accuracy", np.mean(pred_ys == ys))

def training_experiment(name, featurizer, n_train):
    print(f"{name} features, {n_train} examples")
    train_xs = np.array([
        hmm_featurizer(tokenizer.tokenize(review)) 
        for review in train_reviews[:n_train]
    ])
    train_ys = train_labels[:n_train]
    test_xs = np.array([
        hmm_featurizer(tokenizer.tokenize(review))
        for review in test_reviews
    ])
    test_ys = test_labels
    model = train_model(train_xs, train_ys)
    eval_model(model, test_xs, test_ys)
    print()

def hmm_featurizer(review):
    flat_review = []
    for i in review:
        #print(i)
        if i != []:
            flat_review.append(i[0])
            
    _, _, gamma = hmm.forward_backward(flat_review)
    return gamma.sum(axis=0)

training_experiment("hmm", hmm_featurizer, n_train=3000)

hmm features, 3000 examples
test accuracy 0.618



/Users/mingye/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


**Part 3: Lab writeup**

1. What do the learned hidden states seem to encode when you run unsupervised 
   HMM training with only 2 states? What about 10? What about 100?

2. As before, what's the relationship between # of labeled examples and    
   usefulness of HMM-based sentence representations? Are these results generally
   better or worse than in Parts 1 and 2 of the homework? Why or why not might
   HMM state distributions be sensible sentence representations?